In [11]:
import os
from openai import OpenAI

client = OpenAI(
    api_key= "sk-b0a86ccd9a3d48e59a2ba55dccbbfb7a",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# model = client.models
# for m in model.list():
#     print(m)

completion = client.chat.completions.create(
    model="qwen-plus", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {'role': 'system', 'content': '你是一个python大师.'},
        {'role': 'user', 'content': '一般有哪些优化方法'}
    ]
)
print(completion.choices[0].message.content)
print(completion)

Model(id='qwen-vl-ocr-latest', created=1732693133, object='model', owned_by='system')
Model(id='qwen-vl-ocr', created=1732693123, object='model', owned_by='system')
Model(id='qwen-coder-plus-1106', created=1731253544, object='model', owned_by='system')
Model(id='qwen-coder-plus', created=1731253525, object='model', owned_by='system')
Model(id='qwen-coder-plus-latest', created=1731253510, object='model', owned_by='system')
Model(id='qwen2.5-coder-3b-instruct', created=1731253500, object='model', owned_by='system')
Model(id='qwen2.5-coder-0.5b-instruct', created=1731253490, object='model', owned_by='system')
Model(id='qwen2.5-coder-14b-instruct', created=1731253478, object='model', owned_by='system')
Model(id='qwen2.5-coder-32b-instruct', created=1731253464, object='model', owned_by='system')
Model(id='qwen-turbo-0919', created=1726638000, object='model', owned_by='system')
Model(id='qwen2.5-0.5b-instruct', created=1726638000, object='model', owned_by='system')
Model(id='qwen2.5-1.5b-ins